## Configuração Inicial

In [ ]:
import os
import subprocess
import time
from datetime import datetime
from PIL import Image
import pandas as pd

# Verificar se está rodando no Google Colab
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/gdrive')
    BASE_DIR = "/content/gdrive/MyDrive/final-project-paralelizacao-tecnica-de-meios-tons"
else:
    BASE_DIR = os.path.expanduser(".")

# Mudar para o diretório base
os.chdir(BASE_DIR)
print(f"Diretório atual: {os.getcwd()}")


Diretório atual: /home/raphael/Desktop/MC970/final-project-paralelizacao-tecnica-de-meios-tons


## Configurações do Projeto

In [2]:
class ProjectConfig:
    def __init__(self):
        self.exercise_dir = BASE_DIR
        self.build_dir = os.path.join(self.exercise_dir, "build")
        self.input_base_dir = os.path.join(self.exercise_dir, "img", "ppm")
        self.output_base_dir = os.path.join(self.exercise_dir, "out")
        self.log_dir = os.path.join(self.exercise_dir, "logs")
        
        # Criar diretórios necessários
        os.makedirs(self.log_dir, exist_ok=True)
        os.makedirs(self.output_base_dir, exist_ok=True)
        
        # Configurações dos executáveis
        self.executables = [
            {
            "name": "serial_stochastic",
            "path": os.path.join(self.build_dir, "serial_cpp"),
            "command_template": "{exec_path} {input} {output} {p} {stochastic} {grayscale_flag}",
            "output_suffix": "_serial_cpp.ppm",
            "params": {"stochastic": 1}
            },
            {
            "name": "serial_no_stochastic",
            "path": os.path.join(self.build_dir, "serial_cpp"),
            "command_template": "{exec_path} {input} {output} {p} {stochastic} {grayscale_flag}",
            "output_suffix": "_serial_cpp.ppm",
            "params": {"stochastic": 0}
            },
            {
            "name": "openmp_stochastic",
            "path": os.path.join(self.build_dir, "omp"),
            "command_template": "{exec_path} {input} {output} {p} {stochastic} {grayscale_flag}",
            "output_suffix": "_openmp.ppm",
            "params": {"stochastic": 1}
            },
            {
            "name": "openmp_no_stochastic",
            "path": os.path.join(self.build_dir, "omp"),
            "command_template": "{exec_path} {input} {output} {p} {stochastic} {grayscale_flag}",
            "output_suffix": "_openmp_no_stochastic.ppm",
            "params": {"stochastic": 0}
            },
            {
            "name": "cuda_stochastic",
            "path": os.path.join(self.build_dir, "cuda"),
            "command_template": "{exec_path} {input} {output} {p} {stochastic} {grayscale_flag}",
            "output_suffix": "_cuda.ppm",
            "params": {"stochastic": 1}
            },
            {
            "name": "cuda_no_stochastic",
            "path": os.path.join(self.build_dir, "cuda"),
            "command_template": "{exec_path} {input} {output} {p} {stochastic} {grayscale_flag}",
            "output_suffix": "_cuda_no_stochastic.ppm",
            "params": {"stochastic": 0}
            }
        ]
        
        # Parâmetros comuns
        self.common_params = {
            "p": 0.5,
            "stochastic": 1,
            "grayscale_flag": "-g"
        }
    
    def get_timestamp(self):
        return datetime.now().strftime('%Y%m%d_%H%M%S')

# Inicializar configurações
config = ProjectConfig()

## Utilitários

In [3]:
class ProjectUtils:
    def __init__(self, config):
        self.config = config
        self.timestamp = config.get_timestamp()
        self.log_filepath = os.path.join(config.log_dir, f"execution_log_{self.timestamp}.txt")
        self.times_filepath = os.path.join(config.log_dir, f"execution_times_{self.timestamp}.csv")
        
        # Inicializar arquivo de log
        with open(self.log_filepath, 'w') as log_file:
            log_file.write("Log de Execução - Comparação de Implementações\n")
            log_file.write(f"Data: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
    
    def run_command(self, command, work_dir=None):
        """Executa um comando de shell e retorna o resultado."""
        if work_dir is None:
            work_dir = self.config.exercise_dir
            
        return subprocess.run(
            command, shell=True, cwd=work_dir,
            text=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE
        )
    
    def get_image_dimensions(self, image_path):
        """Obtém as dimensões de uma imagem."""
        try:
            with Image.open(image_path) as img:
                return img.size  # (width, height)
        except Exception as e:
            self.log_message(f"Erro ao obter dimensões de {image_path}: {str(e)}", False)
            return (0, 0)
    
    def log_message(self, message, print_to_console=True):
        """Registra uma mensagem no log e opcionalmente no console."""
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        log_entry = f"[{timestamp}] {message}\n"
        
        with open(self.log_filepath, 'a') as log_file:
            log_file.write(log_entry)
        
        if print_to_console:
            print(message)
    
    def find_test_images(self):
        """Encontra todas as imagens ppm no diretório de entrada."""
        test_images = []
        for root, dirs, files in os.walk(self.config.input_base_dir):
            for file in files:
                if file.endswith(".ppm"):
                    rel_path = os.path.relpath(os.path.join(root, file), self.config.input_base_dir)
                    test_images.append(rel_path)
        return test_images
    
    def generate_report(self, times_df):
        """Gera um relatório comparativo de desempenho."""
        report_df = times_df.groupby(['Implementacao', 'Largura', 'Altura'])['Tempo(s)'].mean().unstack(level=0)
        report_filepath = os.path.join(self.config.log_dir, f"performance_report_{self.timestamp}.txt")
        
        with open(report_filepath, 'w') as report_file:
            report_file.write("=== TEMPOS DE EXECUÇÃO POR IMPLEMENTAÇÃO ===\n")
            report_file.write(report_df.to_string())
            
            # Calcular speedup em relação ao serial C
            if 'serial_cpp' in report_df.columns:
                speedup_df = report_df.copy()
                for col in speedup_df.columns:
                    if col != 'serial_cpp':
                        speedup_df[col] = speedup_df['serial_cpp'] / speedup_df[col]
                
                report_file.write("\n\n=== SPEEDUP EM RELAÇÃO AO SERIAL CPP ===\n")
                report_file.write(speedup_df.drop(columns=['serial_cpp']).to_string())
        
        return report_filepath

# Inicializar utilitários
utils = ProjectUtils(config)

## Compilação do Projeto

In [4]:
def build_project():
    """Compila o projeto usando CMake e make."""
    utils.log_message("Limpando diretório de build...")
    utils.run_command("cmake -E remove_directory build")
    utils.run_command("cmake -E make_directory build")
    
    print("Caminho atual:", os.getcwd())
    
    # %cd build/

    # Executar CMake
    utils.log_message("Executando CMake...")
    cmake_result = utils.run_command("cmake ..", config.build_dir)
    utils.log_message(f"CMAKE STDOUT:\n{cmake_result.stdout}", False)
    utils.log_message(f"CMAKE STDERR:\n{cmake_result.stderr}", False)
    assert cmake_result.returncode == 0, "Erro na configuração com CMake"
    
    # Compilar com make
    utils.log_message("Compilando com make...")
    make_result = utils.run_command("make -j", config.build_dir)
    utils.log_message(f"MAKE STDOUT:\n{make_result.stdout}", False)
    utils.log_message(f"MAKE STDERR:\n{make_result.stderr}", False)
    if make_result.returncode != 0:
        print("Erro na compilação (make):")
        print(make_result.stderr)
        assert False, "Erro na compilação (make)"
    
    utils.log_message("Compilação concluída com sucesso!")

# Compilar o projeto
build_project()

Limpando diretório de build...
Caminho atual: /home/raphael/Desktop/MC970/final-project-paralelizacao-tecnica-de-meios-tons
Executando CMake...
Compilando com make...
Compilação concluída com sucesso!


## Processamento das Imagens

In [10]:
def process_images():
    """Processa todas as imagens com as diferentes implementações."""
    # Encontrar imagens para processar
    test_images = utils.find_test_images()
    utils.log_message(f"Total de imagens encontradas: {len(test_images)}")
    
    # DataFrame para armazenar os tempos
    times_data = []
    
    # Processar imagens
    for i, rel_image_path in enumerate(test_images, start=1):
        img_path = os.path.join(config.input_base_dir, rel_image_path)
        dimensions = utils.get_image_dimensions(img_path)
        size_str = f"{dimensions[0]}x{dimensions[1]}"
        
        utils.log_message(f"\nProcessando imagem {i}/{len(test_images)}: {rel_image_path} ({size_str})")
        
        for exec_config in config.executables:
            exec_name = exec_config["name"]
            exec_path = exec_config["path"]
            
            # Criar diretório de saída
            output_dir = os.path.join(config.output_base_dir, exec_name, os.path.dirname(rel_image_path))
            os.makedirs(output_dir, exist_ok=True)
            
            # Nome do arquivo de saída
            output_filename = os.path.basename(rel_image_path).replace(".ppm", exec_config["output_suffix"])
            output_path = os.path.join(output_dir, output_filename)
            
            # Preparar parâmetros
            params = {**config.common_params, **exec_config.get("params", {})}
            params.update({
                "exec_path": exec_path,
                "input": img_path,
                "output": output_path
            })
            
            # Construir comando
            command = exec_config["command_template"].format(**params)
            
            utils.log_message(f"  Executando {exec_name}...")
            
            # Executar e medir tempo
            start_time = time.time()
            result = subprocess.run(
                command, shell=True, cwd=config.exercise_dir,
                stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                text=True
            )
            # Extrair tempo de execução da saída do programa
            stdout = result.stdout
            execution_time = None

            if exec_name in ["serial_cpp", "serial_stochastic", "serial_no_stochastic"]:
                # Exemplo: "Serial execution time: 96.61 ms"
                for line in stdout.splitlines():
                    if "Serial execution time:" in line:
                        try:
                            ms = float(line.split(":")[1].strip().split()[0])
                            execution_time = ms / 1000.0
                        except Exception:
                            pass
            elif exec_name in ["openmp_stochastic", "openmp_no_stochastic"]:
                # Exemplo: "OpenMP execution time (2 threads): 96.1256 ms"
                for line in stdout.splitlines():
                    if "OpenMP execution time" in line:
                        try:
                            ms = float(line.split(":")[1].strip().split()[0])
                            execution_time = ms / 1000.0
                        except Exception:
                            pass
            elif exec_name in ["cuda_stochastic", "cuda_no_stochastic"]:
                # Exemplo: "CUDA execution time: 1.60691 ms" ou "Execution time: 1.66567 ms"
                for line in stdout.splitlines():
                    if "CUDA execution time:" in line or "Execution time:" in line:
                        try:
                            ms = float(line.split(":")[1].strip().split()[0])
                            execution_time = ms / 1000.0
                        except Exception:
                            pass
            


            
            # Fallback: se não encontrou, usa medição de tempo Python
            if execution_time is None:
                execution_time = time.time() - start_time
                print(f"Falha ao extrair tempo de execução do {exec_name} para a imagem {rel_image_path}. Usando tempo medido em Python: {execution_time:.4f}s")


            # Registrar resultados
            times_data.append({
                "Imagem": rel_image_path,
                "Largura": dimensions[0],
                "Altura": dimensions[1],
                "Implementacao": exec_name,
                "Tempo(s)": execution_time,
                "Output_Path": output_path
            })
            
            # Log de execução
            if result.returncode != 0:
                utils.log_message(f"  ERRO em {exec_name}: {result.stderr}", False)
            else:
                utils.log_message(f"  {exec_name} concluído em {execution_time:.4f}s")
    
    # Salvar tempos em CSV
    times_df = pd.DataFrame(times_data)
    times_df.to_csv(utils.times_filepath, index=False)
    
    # Gerar relatório comparativo
    report_filepath = utils.generate_report(times_df)
    
    utils.log_message(f"\nProcessamento concluído. Resultados salvos em:")
    utils.log_message(f"- Log completo: {utils.log_filepath}")
    utils.log_message(f"- Tempos de execução: {utils.times_filepath}")
    utils.log_message(f"- Relatório comparativo: {report_filepath}")
    
    return times_df

# Processar as imagens
results_df = process_images()

Total de imagens encontradas: 22

Processando imagem 1/22: large/large_1.ppm (1024x640)
  Executando serial_stochastic...
  serial_stochastic concluído em 0.1684s
  Executando serial_no_stochastic...
  serial_no_stochastic concluído em 0.0094s
  Executando openmp_stochastic...
  openmp_stochastic concluído em 0.0304s
  Executando openmp_no_stochastic...
  openmp_no_stochastic concluído em 0.0169s
  Executando cuda_stochastic...
  cuda_stochastic concluído em 0.0004s
  Executando cuda_no_stochastic...
  cuda_no_stochastic concluído em 0.0869s

Processando imagem 2/22: large/large_2.ppm (640x426)
  Executando serial_stochastic...
  serial_stochastic concluído em 0.0624s
  Executando serial_no_stochastic...
  serial_no_stochastic concluído em 0.0035s
  Executando openmp_stochastic...
  openmp_stochastic concluído em 0.0185s
  Executando openmp_no_stochastic...
  openmp_no_stochastic concluído em 0.0044s
  Executando cuda_stochastic...
  cuda_stochastic concluído em 0.0003s
  Executando cu

## Análise dos Resultados

In [30]:
def analyze_results(results_df):
    """Realiza análise básica dos resultados."""
    # Exibir estatísticas básicas
    print("\n=== ESTATÍSTICAS BÁSICAS ===")
    print(results_df.groupby('Implementacao')['Tempo(ms)'].describe())
    
    # Calcular speedup médio em relação ao serial C
    if 'serial_no_stochastic' in results_df['Implementacao'].unique():
        print("\n=== SPEEDUP MÉDIO ===")
        serial_times = results_df[results_df['Implementacao'] == 'serial_no_stochastic'].set_index(['Largura', 'Altura'])['Tempo(s)']
        
        for impl in results_df['Implementacao'].unique():
            if impl != 'serial_no_stochastic':
                impl_times = results_df[results_df['Implementacao'] == impl].set_index(['Largura', 'Altura'])['Tempo(s)']
                speedup = (serial_times / impl_times).mean()
                print(f"{impl}: {speedup:.2f}x mais rápido que serial_cpp")

import re

def filter_results_by_size(results_df, size):
    """Filtra os resultados por tamanho de imagem."""
    print(f"\n=== Resultados para imagens {size.upper()} ===")
    df_size = results_df[results_df['Imagem'].str.match(f"^{size}/")]
    for impl in results_df['Implementacao'].unique():
        df_impl = df_size[df_size['Implementacao'] == impl]
        if not df_impl.empty:
            print(f"\nImplementação: {impl}")
            print(df_impl[['Imagem', 'Tempo(ms)']].to_string(index=False))
            print((df_impl['Tempo(ms)'].describe()).round(2))

# Analisar resultados
analyze_results(results_df)

# Filtrar por tamanho de imagem e implementação
results_df['Tempo(ms)'] = (results_df['Tempo(s)'] * 1000).round(2)
# for size in ['small', 'medium', 'large', 'xlarge']:
#     filter_results_by_size(results_df, size)
    
# display(results_df)


=== ESTATÍSTICAS BÁSICAS ===
                      count        mean         std    min      25%     50%  \
Implementacao                                                                 
cuda_no_stochastic     22.0  150.434091  324.046001   8.41  34.7450  37.250   
cuda_stochastic        22.0    0.451364    0.316262   0.29   0.3200   0.350   
openmp_no_stochastic   22.0   11.918182   15.307086   0.28   5.3625   6.915   
openmp_stochastic      22.0   57.847273  113.184633   1.68  16.1275  19.110   
serial_no_stochastic   22.0   21.750909   51.945629   0.66   3.5025   3.865   
serial_stochastic      22.0  360.797273  857.816022  11.56  60.4125  63.495   

                           75%      max  
Implementacao                            
cuda_no_stochastic    123.5800  1556.67  
cuda_stochastic         0.4475     1.80  
openmp_no_stochastic   14.6750    75.41  
openmp_stochastic      51.1725   545.25  
serial_no_stochastic   15.1075   247.56  
serial_stochastic     263.5625  4097.33  

